In [1]:
import requests
import pandas as pd
import numpy as np
import json
import pprint
import io
from datetime import date, timedelta
import json
import urllib3
import re
import gspread
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
from unidecode import unidecode
from oauth2client.service_account import ServiceAccountCredentials
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import cx_Oracle
import sys
from requests_oauthlib import OAuth1
import requests

yester_day_fun = date.today() - timedelta(days=1)
ddc_day_fun = date.today() - timedelta(days=2)
ddc_day_fun3 = date.today() - timedelta(days=3)
today_fun = date.today()
last_week_fun = date.today() - timedelta(days=7)
window_fun2 = date.today() - timedelta(days=30)
window_fun = pd.Timestamp('today').floor('D') - timedelta(days=30)

In [2]:
#CONFIGURANDO A API APPSFLYER
#DEFININDO AS VARIÁVEIS DE DATA, IDS DOS APPS E FORMATO DOS REPORTS
yester_day_fun = date.today() - timedelta(days=1)
last_week_fun = date.today() - timedelta(days=7)
android_id = ''
ios_id = ''
installs_rep = 'installs_report'
inapp_rep = 'in_app_events_report'
org_installs_rep = 'organic_installs_report'
org_inapp_rep = 'organic_in_app_events_report'
uninstalls_rep = 'uninstall_events_report'
org_uninstalls_rep = 'organic_uninstall_events_report'

#DEFINIÇÃO DOS PARÂMETROS PRA CHAMADA
params = {
  'api_token': '',
  'from': ddc_day_fun3,
  'to': yester_day_fun,
  'maximum_rows': '1000000'
}

#FORMATANDO OS LINKS
req_installs_android = 'https://hq.appsflyer.com/export/{}/{}/v5'.format(android_id, installs_rep)
req_installs_ios = 'https://hq.appsflyer.com/export/{}/{}/v5'.format(ios_id, installs_rep)
req_inapp_android = 'https://hq.appsflyer.com/export/{}/{}/v5'.format(android_id, inapp_rep)
req_inapp_ios = 'https://hq.appsflyer.com/export/{}/{}/v5'.format(ios_id, inapp_rep)
req_org_installs_android = 'https://hq.appsflyer.com/export/{}/{}/v5'.format(android_id, org_installs_rep)
req_org_installs_ios = 'https://hq.appsflyer.com/export/{}/{}/v5'.format(ios_id, org_installs_rep)
req_org_inapp_android = 'https://hq.appsflyer.com/export/{}/{}/v5'.format(android_id, org_inapp_rep)
req_org_inapp_ios = 'https://hq.appsflyer.com/export/{}/{}/v5'.format(ios_id, org_inapp_rep)
req_uninstalls_android = 'https://hq.appsflyer.com/export/{}/{}/v5'.format(android_id, uninstalls_rep)
req_uninstalls_ios = 'https://hq.appsflyer.com/export/{}/{}/v5'.format(ios_id, uninstalls_rep)
req_org_uninstalls_android = 'https://hq.appsflyer.com/export/{}/{}/v5'.format(android_id, org_uninstalls_rep)
req_org_uninstalls_ios = 'https://hq.appsflyer.com/export/{}/{}/v5'.format(ios_id, org_uninstalls_rep)

#FAZENDO A REQUEST DOS DADOS PELO GET
res_installs_android = requests.request('GET', req_installs_android, params=params)
res_installs_ios = requests.request('GET', req_installs_ios, params=params)
res_inapp_android = requests.request('GET', req_inapp_android, params=params)
res_inapp_ios = requests.request('GET', req_inapp_ios, params=params)
res_org_installs_android = requests.request('GET', req_org_installs_android, params=params)
res_org_installs_ios = requests.request('GET', req_org_installs_ios, params=params)
res_org_inapp_android = requests.request('GET', req_org_inapp_android, params=params)
res_org_inapp_ios = requests.request('GET', req_org_inapp_ios, params=params)
res_uninstalls_android = requests.request('GET', req_uninstalls_android, params=params)
res_uninstalls_ios = requests.request('GET', req_uninstalls_ios, params=params)
res_org_uninstalls_android = requests.request('GET', req_org_uninstalls_android, params=params)
res_org_uninstalls_ios = requests.request('GET', req_org_uninstalls_ios, params=params)

In [3]:
#TRATANDO AS EXCEÇÕES CASO A CALL SEJA PROBLEMÁTICA
if res_uninstalls_android.status_code != 200:
  if res_uninstalls_android.status_code == 404:
    print('There is a problem with Uninstalls Android request URL. Make sure that it is correct')
  else:
    print('There was a problem retrieving data: ', res_uninstalls_android.text)
else:
    df_uninstalls_android = pd.read_csv(io.StringIO(res_uninstalls_android.text), low_memory=False)
        
if res_uninstalls_ios.status_code != 200:
  if res_uninstalls_ios.status_code == 404:
    print('There is a problem with Uninstalls iOS request URL. Make sure that it is correct')
  else:
    print('There was a problem retrieving data: ', res_uninstalls_ios.text)
else:
    df_uninstalls_ios = pd.read_csv(io.StringIO(res_uninstalls_ios.text), low_memory=False)
        
if res_org_uninstalls_android.status_code != 200:
  if res_org_uninstalls_android.status_code == 404:
    print('There is a problem with Organic Uninstalls Android request URL. Make sure that it is correct')
  else:
    print('There was a problem retrieving data: ', res_org_uninstalls_android.text)
else:
    df_org_uninstalls_android = pd.read_csv(io.StringIO(res_org_uninstalls_android.text), low_memory=False)
        
if res_org_uninstalls_ios.status_code != 200:
  if res_org_uninstalls_ios.status_code == 404:
    print('There is a problem with Organic Uninstalls iOS request URL. Make sure that it is correct')
  else:
    print('There was a problem retrieving data: ', res_org_uninstalls_ios.text)
else:
    df_org_uninstalls_ios = pd.read_csv(io.StringIO(res_org_uninstalls_ios.text), low_memory=False)

In [4]:
if res_inapp_android.status_code != 200:
  if res_inapp_android.status_code == 404:
    print('There is a problem with InApp Android request URL. Make sure that it is correct')
  else:
    print('There was a problem retrieving data: ', res_inapp_android.text)
else:
    df_inapp_android = pd.read_csv(io.StringIO(res_inapp_android.text), low_memory=False)
        
if res_inapp_ios.status_code != 200:
  if res_inapp_ios.status_code == 404:
    print('There is a problem with InApp iOS request URL. Make sure that it is correct')
  else:
    print('There was a problem retrieving data: ', res_inapp_ios.text)
else:
    df_inapp_ios = pd.read_csv(io.StringIO(res_inapp_ios.text), low_memory=False)
        
if res_org_inapp_android.status_code != 200:
  if res_org_inapp_android.status_code == 404:
    print('There is a problem with Organic InApp Android request URL. Make sure that it is correct')
  else:
    print('There was a problem retrieving data: ', res_org_inapp_android.text)
else:
    df_org_inapp_android = pd.read_csv(io.StringIO(res_org_inapp_android.text), low_memory=False)
        
if res_org_inapp_ios.status_code != 200:
  if res_org_inapp_ios.status_code == 404:
    print('There is a problem with Organic InApp iOS request URL. Make sure that it is correct')
  else:
    print('There was a problem retrieving data: ', res_org_inapp_ios.text)
else:
    df_org_inapp_ios = pd.read_csv(io.StringIO(res_org_inapp_ios.text), low_memory=False)

In [5]:
if res_installs_android.status_code != 200:
  if res_installs_android.status_code == 404:
    print('There is a problem with Installs Android request URL. Make sure that it is correct')
  else:
    print('There was a problem retrieving data: ', res_installs_android.text)
else:
    df_installs_android = pd.read_csv(io.StringIO(res_installs_android.text), low_memory=False)
        
if res_installs_ios.status_code != 200:
  if res_installs_ios.status_code == 404:
    print('There is a problem with Installs iOS request URL. Make sure that it is correct')
  else:
    print('There was a problem retrieving data: ', res_installs_ios.text)
else:
    df_installs_ios = pd.read_csv(io.StringIO(res_installs_ios.text), low_memory=False)
        
if res_org_installs_android.status_code != 200:
  if res_org_installs_android.status_code == 404:
    print('There is a problem with Organic Installs Android request URL. Make sure that it is correct')
  else:
    print('There was a problem retrieving data: ', res_org_installs_android.text)
else:
    df_org_installs_android = pd.read_csv(io.StringIO(res_org_installs_android.text), low_memory=False)
        
if res_org_installs_ios.status_code != 200:
  if res_org_installs_ios.status_code == 404:
    print('There is a problem with Organic Installs iOS request URL. Make sure that it is correct')
  else:
    print('There was a problem retrieving data: ', res_org_installs_ios.text)
else:
    df_org_installs_ios = pd.read_csv(io.StringIO(res_org_installs_ios.text), low_memory=False)

In [6]:
#DISCLOSED
#DEFININDO CREDENCIAIS PARA EXTRAÇÃO VIA QUERIES NO ORACLE
cx_Oracle.init_oracle_client(lib_dir="C:\oracle\instantclient_19_8", config_dir="C:\oracle\instantclient_19_8")

pw = ""
conn = cx_Oracle.connect("", pw, "", encoding="UTF-8")
c = conn.cursor()

In [1]:
#DISCLOSED
#DEFININDO QUERIES NO ORACLE
clientes_oracle = '''SELECT V_CUSTOMER_ALL.ID_CUSTOMER "ID", V_CUSTOMER_ALL.CUSTOMER "Cliente", V_CUSTOMER_ALL.TELEPHONE "Telefone", V_CUSTOMER_ALL.EMAIL "Email", V_CUSTOMER_ALL.ACCEPTED_TERM_DATE "Criação", V_CUSTOMER_ALL.CUSTOMER_LAST_UPDATE_DATE "Última Interação", V_CUSTOMER_ALL.STORE_RATING "Avaliação Loja", V_CUSTOMER_ALL.ACCEPTED_TERM_DATE "Aceite Termos", V_CUSTOMER_ALL.SUBSCRIPTION_CANCEL_DATE "Cancelamento Assinatura", V_CUSTOMER_ALL.CHANNEL "Canal", V_CUSTOMER_ALL.ADDRESS "Endereço", V_CUSTOMER_ALL.ADDRESS_NUMBER "Nº", V_CUSTOMER_ALL.COMPLEMENT "Complemento", V_CUSTOMER_ALL.NEIGHBORHOOD "Bairro", V_CUSTOMER_ALL.UF, V_CUSTOMER_ALL.CITY "Cidade", V_CUSTOMER_ALL.ADDRESS_MAIN "Principal", V_CUSTOMER_ALL.REFERENCE "Referência", V_CUSTOMER_ALL.QTD_PEDIDO "Qtd Pedidos", V_CUSTOMER_ALL.ULT_PEDIDO "Último Pedido", V_CUSTOMER_ALL.QTD_INTERACAO "Qtd Interação", V_CUSTOMER_ALL.QTD_PESQUISA "Qtd Pesquisas", V_CUSTOMER_ALL.ULT_PESQUISA "Última Pesquisa", V_CUSTOMER_ALL.AUTH_TOKEN "Auth Token"
FROM V_CUSTOMER_ALL
WHERE V_CUSTOMER_ALL.CHANNEL = 'App' '''

sql_clientes = pd.read_sql_query(clientes_oracle, conn)

sql_clientes

NameError: name 'pd' is not defined

In [8]:
#DISCLOSED
#DEFININDO CREDENCIAIS PARA CONSUMO DA API SFMC
#Token:
token = "token"

{   
"client_id": "",
"client_secret": "",
"grant_type": "client_credentials"
}

grant_type = 'client_credentials'
body_params = {'grant_type' : grant_type}

client_id = ""
client_secret = ""

get_url_recompra_push = 'https://url1.rest.marketingcloudapis.com/'
get_url_recompra_email = 'https://url2.rest.marketingcloudapis.com/'

get_url_geral_push = 'https://url3.rest.marketingcloudapis.com/'
get_url_geral_email = 'https://url4.rest.marketingcloudapis.com/'

In [2]:
#DISCLOSED
client_id = ""
client_secret = ""
grant_type = 'client_credentials'

response_mktcloud = requests.post(token, auth=(client_id, client_secret),
                         data={'grant_type':grant_type,'client_id':client_id,'client_secret':client_secret})

print(response_mktcloud.json())

NameError: name 'requests' is not defined

In [10]:
tok_dict = json.loads(response_mktcloud.text)
access_token = tok_dict['access_token']

In [11]:
r_recompra_push = requests.get(get_url_recompra_push, headers = {"Authorization":"Bearer " + access_token})
r_recompra_email = requests.get(get_url_recompra_email, headers = {"Authorization":"Bearer " + access_token})

In [12]:
r_geral_push = requests.get(get_url_geral_push, headers = {"Authorization":"Bearer " + access_token})
r_geral_email = requests.get(get_url_geral_email, headers = {"Authorization":"Bearer " + access_token})

In [3]:
#DISCLOSED
data_geral_push = r_geral_push.json()
df_geral_push = pd.json_normalize(data_geral_push, 'items')
df_geral_push.head(10)

NameError: name 'r_geral_push' is not defined

In [14]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
#INÍCIO DA ANÁLISE EXPLORATÓRIA

In [15]:
df_geral_push['keys.messagename'].value_counts()

MP_DOMICILIAR_REGUA_CANAIS_RECOMPRA_45                  243
MP_DOMICILIAR_REGUA_CANAIS_RECOMPRA_35                  235
MP_DOMICILIAR_REGUA_CANAIS_RECOMPRA_30                  225
MP_DOMICILIAR_REGUA_CANAIS_RECOMPRA_25                  216
MP_REGUA_CANAIS_ABANDONO_GERAL                          215
MP_REGUA_CANAIS_CADASTRO_GERAL                          213
MP_DOMICILIAR_PRIMEIRA_COMPRA_5_PORCENTO                 55
MP_DOMICILIAR_PRIMEIRA_COMPRA_10_PORCENTO                55
MP_DOMICILIAR_INCISIVO_RECOMPRA_5_PORCENTO               54
MP_DOMICILIAR_INCISIVO_RECOMPRA_10_PORCENTO              54
PUSH_DOMICILIAR_JORNAD_ABANDONONAVEG_1H_B_10OFF          36
PUSH_DOMICILIAR_JORNAD_CARRINHOABANDONADO_1H_B_5OFF      36
PUSH_DOMICILIAR_JORNAD_CARRINHOABANDONADO_1H_B_10OFF     36
PUSH_DOMICILIAR_JORNAD_ABANDONONAVEG_1H_B_5OFF           36
MP_DOMICILIAR_REGUA_ABANDONONAVEG1D                      23
MP_REGUA_CANAIS_CADASTRO_SC_SP                           18
MP_MERCADO_SUL_ABANDONO                 

In [16]:
df_geral_push['keys.data_envio'].value_counts().sort_index()

20200909       1
20200910       4
20200914       3
20200915       4
20200916       3
20201006       1
20201007       1
20201008       3
20201009       4
20201013       1
20201014       4
20201015       2
20201016       2
20201019       2
20201020       3
20201021       3
20201022       4
20201023       4
20201026       5
20201027       3
20201028       4
20201029       4
20201030       5
20201103       4
20201104       5
20201105       4
20201106       4
20201109       2
20201110       5
20201111       5
20201112       3
20201113       3
20201130       4
20201203       1
20201211       6
20201212       4
20201213       4
20201214       6
20201215       6
20201216       6
20201217       7
20201218       6
20201219       4
20201220       4
20201221       6
20201222       5
20201223       7
20201224       6
20201225       6
20201226       3
20201227       2
20201228       6
20201229       6
20201230       6
20210101       3
20210102       4
20210103       4
20210104       6
20210105      

In [4]:
#DISCLOSED
data_geral_email = r_geral_email.json()
df_geral_email = pd.json_normalize(data_geral_email, 'items')
df_geral_email.head(10)

NameError: name 'r_geral_email' is not defined

In [18]:
df_geral_email['values.emailname'].value_counts()

EM_REVENDA_PESQ_PRE_VENDA_JB                                            246
EM_REVENDA_PESQ_POS_CONSULTOR_V2                                        210
EM_EMP_MANUT_CORR_PROTOCOLO                                             203
EM_DOMICILIAR_REGUA_CANAIS_RECOMPRA_45                                  202
EM_DOMICILIAR_REGUA_CANAIS_RECOMPRA_35                                  197
EM_EMP_MANUT_CORR_AGENDAMENTO                                           192
EM_DOMICILIAR_REGUA_CANAIS_RECOMPRA_30                                  190
EM_DOMICILIAR_REGUA_CANAIS_RECOMPRA_25                                  181
EM_EMP_MANUT_CORR_MUD_TECNICO                                           126
EM_MI_REGUACOBRANÇA_D7_NOME                                              93
EM_EMP_MANUT_CORR_PESQ_SATIS                                             66
EM_DOMICILIAR_PRIMEIRA_COMPRA_INCICIVO_REFORCO_10_PORCENTO               49
EM_DOMICILIAR_PRIMEIRA_COMPRA_5_PORCENTO                                 49
EM_DOMICILIA

In [5]:
#DISCLOSED
data_recompra_push = r_recompra_push.json()
df_recompra_push = pd.json_normalize(data_recompra_push, 'items')
df_recompra_push.head(10)

NameError: name 'r_recompra_push' is not defined

In [6]:
#DISCLOSED
data_recompra_email = r_recompra_email.json()
df_recompra_email = pd.json_normalize(data_recompra_email, 'items')
df_recompra_email.head(10)

NameError: name 'r_recompra_email' is not defined

In [ ]:
#MODELAGEM DOS DATAFRAMES PARA INCLUSÃO NO NOVO MODELO DE DBS

In [21]:
df_recompra_push['refined.data_envio'] = pd.to_datetime(df_recompra_push['keys.data_envio'])
df_recompra_email['refined.data_envio'] = pd.to_datetime(df_recompra_email['keys.data_envio'])

In [23]:
sql_clientes_ativos = sql_clientes[sql_clientes['Última Interação']<window_fun]

In [24]:
df_uninstalls_androidTotal = df_uninstalls_android.append(df_org_uninstalls_android)
df_uninstalls_iosTotal = df_uninstalls_ios.append(df_org_uninstalls_ios)
df_uninstalls_final = df_uninstalls_androidTotal.append(df_uninstalls_iosTotal)

In [25]:
df_installs_androidTotal = df_installs_android.append(df_org_installs_android)
df_installs_iosTotal = df_installs_ios.append(df_org_installs_ios)
df_installs_final = df_installs_androidTotal.append(df_installs_iosTotal)

In [26]:
df_inapp_androidTotal = df_inapp_android.append(df_org_inapp_android)
df_inapp_iosTotal = df_inapp_ios.append(df_org_inapp_ios)
df_inapp_final = df_inapp_androidTotal.append(df_inapp_iosTotal)

In [27]:
df_inapp_final = df_inapp_final.rename(columns={'Customer User ID': 'ID'})
df_inapp_final['Media Source'].fillna("Orgânico", inplace=True)

In [28]:
dict_inapp = df_inapp_final[['ID', 'Media Source']]

In [29]:
df_clientes_null = sql_clientes[sql_clientes['ID'].isna() == False]
df_clientes_ativos_null = sql_clientes_ativos[sql_clientes_ativos['ID'].isna() == False]
df_clientes_null['ID'] = df_clientes_null['ID'].apply(str)
df_clientes_ativos_null['ID'] = df_clientes_ativos_null['ID'].apply(str)

In [30]:
df_clientes_final = pd.merge(df_clientes_null, dict_inapp, on='ID', how='left')
df_clientes_ativos_final = pd.merge(df_clientes_ativos_null, dict_inapp, on='ID', how='left')

In [31]:
pd.options.display.max_columns = 999

In [33]:
df_clientes_final.drop_duplicates(subset=['ID'], inplace=True)

In [ ]:
#REDEFININDO DATATYPE DE ANÁLISE TEMPORAL

In [35]:
def to_datetime():
    df_uninstalls_final['Install Time'] = pd.to_datetime(df_uninstalls_final['Install Time'])
    df_uninstalls_final['Event Time'] = pd.to_datetime(df_uninstalls_final['Event Time'])
    df_installs_final['Install Time'] = pd.to_datetime(df_installs_final['Install Time'])
    df_installs_final['Event Time'] = pd.to_datetime(df_installs_final['Event Time'])
    df_inapp_final['Install Time'] = pd.to_datetime(df_inapp_final['Install Time'])
    df_inapp_final['Event Time'] = pd.to_datetime(df_inapp_final['Event Time'])
    
def new_data_hora():
    df_uninstalls_final['Data Install'] = [d.date() for d in df_uninstalls_final['Install Time']]
    df_uninstalls_final['Hora Install'] = [d.time() for d in df_uninstalls_final['Install Time']]
    df_uninstalls_final['Data Event'] = [d.date() for d in df_uninstalls_final['Event Time']]
    df_uninstalls_final['Hora Event'] = [d.time() for d in df_uninstalls_final['Event Time']]
    df_installs_final['Data Install'] = [d.date() for d in df_installs_final['Install Time']]
    df_installs_final['Hora Install'] = [d.time() for d in df_installs_final['Install Time']]
    df_installs_final['Data Event'] = [d.date() for d in df_installs_final['Event Time']]
    df_installs_final['Hora Event'] = [d.time() for d in df_installs_final['Event Time']]
    df_inapp_final['Data Install'] = [d.date() for d in df_inapp_final['Install Time']]
    df_inapp_final['Hora Install'] = [d.time() for d in df_inapp_final['Install Time']]
    df_inapp_final['Data Event'] = [d.date() for d in df_inapp_final['Event Time']]
    df_inapp_final['Hora Event'] = [d.time() for d in df_inapp_final['Event Time']]

In [36]:
to_datetime()
new_data_hora()

In [37]:
df_uninstalls_final['Event Time'] = pd.to_datetime(df_uninstalls_final['Event Time'],format='%y/%m/%d')
df_installs_final['Event Time'] = pd.to_datetime(df_installs_final['Event Time'],format='%y/%m/%d')
df_inapp_final['Event Time'] = pd.to_datetime(df_inapp_final['Event Time'],format='%y/%m/%d')

In [39]:
df_uninstall_diff = df_uninstalls_final['Data Event'] - df_uninstalls_final['Data Install']
df_uninstall_diff.value_counts()

5 days      6642
4 days      6541
3 days      4938
6 days      4331
2 days      2556
7 days      2054
1 days      1986
8 days       331
9 days       174
11 days      117
10 days      114
0 days       103
12 days       82
13 days       61
56 days       42
53 days       42
14 days       39
54 days       39
55 days       36
52 days       33
51 days       29
57 days       24
61 days       22
58 days       21
15 days       21
20 days       20
47 days       20
19 days       20
31 days       18
16 days       18
50 days       17
17 days       16
60 days       16
63 days       16
71 days       15
59 days       14
18 days       14
48 days       14
62 days       14
46 days       13
65 days       13
23 days       13
34 days       13
27 days       13
67 days       12
64 days       12
38 days       12
42 days       11
40 days       11
111 days      11
30 days       11
28 days       11
22 days       10
82 days       10
72 days       10
100 days      10
26 days       10
66 days       10
35 days       

In [40]:
#pd.set_option('display.max_columns', 500)

In [41]:
df_inapp_append1 = df_inapp_androidTotal.append(df_inapp_iosTotal)

In [46]:
import sqlite3

dbfile = sqlite3.connect(r'C:\Users\User\db.db')
cur = dbfile.cursor()
dbfile.commit()

In [47]:
#CÉLULA PARA CRIAÇÃO DE NOVAS DBS


#cur.execute("""CREATE TABLE 'MKTCloud_Push_Geral' ('keys.data_envio', 'keys.messagename', 'values.enviados_total', 
#'values.entregue_total', 'values.abertura_total', 'values.total_compra', 'values.tipo');""")

#dbfile.commit()

In [48]:
table_name1 = 'New_App_Uninstalls'
table_name2 = 'New_App_Installs'
table_name3 = 'New_App_Inapp'
table_name4 = 'New_App_Clientes'
table_name5 = 'MKTCloud_Push_Recompra'
table_name6 = 'MKTCloud_EMail_Recompra'
table_name7 = 'MKTCloud_Push_Geral'
table_name8 = 'MKTCloud_EMail_Geral'


def append_tables(): 
    
    df_uninstalls_final.to_sql(table_name1, dbfile, if_exists='append', index=False)
    df_installs_final.to_sql(table_name2, dbfile, if_exists='append', index=False)
    df_inapp_final.to_sql(table_name3, dbfile, if_exists='append', index=False)
    df_clientes_final.to_sql(table_name4, dbfile, if_exists='replace', index=False)
    df_recompra_push.to_sql(table_name5, dbfile, if_exists='replace', index=False)
    df_recompra_email.to_sql(table_name6, dbfile, if_exists='replace', index=False)
    df_geral_push.to_sql(table_name7, dbfile, if_exists='replace', index=False)
    df_geral_email.to_sql(table_name8, dbfile, if_exists='replace', index=False)

In [49]:
def delete_content_view():
    cur.execute("""
    DELETE FROM New_App_Inapp
    WHERE "Event Name" = 'af_content_view';""")
    dbfile.commit()

In [50]:
def delete_week():
    cur.execute("""DELETE FROM New_App_Inapp WHERE DATE("Data Event") >= DATE('now', '-7 days')""")
    cur.execute("""DELETE FROM 'New_App_Uninstalls' WHERE DATE("Data Event") >= DATE('now', '-3 days')""")
    cur.execute("""DELETE FROM 'New_App_Installs' WHERE DATE("Data Install") >= DATE('now', '-3 days')""")
    dbfile.commit()

In [51]:
def bases_commas():
    cur.execute("""
    UPDATE 
        MKTCloud_EMail_Geral
    SET
        "values.custo_campanha" = REPLACE("values.custo_campanha", '.',',')
    WHERE
        "values.custo_campanha" IS NOT NULL;""")
    
    cur.execute("""
    UPDATE 
        MKTCloud_Push_Geral
    SET
        "values.custo_campanha" = REPLACE("values.custo_campanha", '.',',')
    WHERE
        "values.custo_campanha" IS NOT NULL;""")
    

In [52]:
df_org_inapp_android['Event Time'].value_counts().sort_index()

2021-11-29 03:26:00    4
2021-11-29 03:26:01    1
2021-11-29 03:26:02    2
2021-11-29 03:26:03    4
2021-11-29 03:26:04    2
                      ..
2021-12-01 23:59:52    1
2021-12-01 23:59:55    2
2021-12-01 23:59:56    2
2021-12-01 23:59:57    4
2021-12-01 23:59:59    2
Name: Event Time, Length: 190091, dtype: int64

In [53]:
df_inapp_final['Data Event'].value_counts().sort_index()

2021-11-29    633419
2021-11-30    431747
2021-12-01    288856
Name: Data Event, dtype: int64

In [54]:
df_installs_final['Data Event'].value_counts().sort_index()

2021-11-29    8020
2021-11-30    6347
2021-12-01    5160
Name: Data Event, dtype: int64

In [55]:
df_uninstalls_final['Data Event'].value_counts().sort_index()

2021-11-29    11488
2021-11-30    11221
2021-12-01     8747
Name: Data Event, dtype: int64

In [56]:
#linebreak1

In [57]:
#linebreak2

In [58]:
#linebreak3

In [59]:
delete_week()
append_tables()
bases_commas()
delete_content_view()

C:\Users\caio.arruda\Anaconda3\lib\site-packages\pandas\core\generic.py:2653: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [60]:
get_ipython().system('jupyter nbconvert --to script APIs_AppsFlyer_Uninstalls.ipynb')

[NbConvertApp] Converting notebook APIs_AppsFlyer_Uninstalls.ipynb to script
[NbConvertApp] Writing 20841 bytes to APIs_AppsFlyer_Uninstalls.py
